In [2]:
import torch.nn as nn
import torch
from tqdm import tqdm
import itertools
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast

In [3]:
dataset = load_dataset('lmqg/qa_wiki_t5_large')

Using custom data configuration default
Found cached dataset qa_wiki_t5_large (C:/Users/billj/.cache/huggingface/datasets/lmqg___qa_wiki_t5_large/default/0.0.0/f3a1d4d9e366c8e5d66c9329ca2a32b4cb673782cda308c7ef928789d431cfcb)
100%|██████████| 2/2 [00:00<00:00,  4.50it/s]


In [4]:
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file = "qa_wiki_t5_large_tokenizer.json")

In [5]:
fast_tokenizer("wikipedia").tokens()

['w', 'ik', 'ipedia']

In [7]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()

        self.w_h = nn.Linear(input_size + hidden_size, hidden_size)
        self.w_o = nn.Linear(hidden_size, input_size)
        self.hidden_size = hidden_size
        self.input_size = input_size
    
    def forward(self, input_token, hidden_state):
        comb = torch.cat([input_token, hidden_state], dim = 1)
        hidden = self.w_h(comb)
        out = self.w_o(hidden)

        return out, hidden


In [8]:
model = RNN(2048, 64)

In [9]:
def convert_to_single_string(sample):
    return sample['title'] + "[SEP]" + sample['context'] + "[ANS]" + sample['answers']['text'][0]

In [ ]:
def train(dataset, model):
    model.train()

    for epoch in range(1):
        for sample in dataset['train']:
            x = convert_to_single_string(sample)
            x = fast_tokenizer(x, return_tensors='pt')['input_ids']
            x = torch.nn.functional.one_hot(x, 2048)

            state_h = torch.zeros(1, 64)
            loss = torch.tensor([0.0])
            for tok_idx in range(x.size()[1] - 1):
                curr_token = x[:, tok_idx+1]
                y_pred, state_h = model(curr_token, state_h)
                y_true = x[:, tok_idx+1]
                loss += crit(y_pred, y_true.float())/(x.size()[1])
                

